# 2021 NYC Municipal Elections Contributions/Expenditures Analysis

## Authorship

Hi 👋! This notebook was built by me. You can find me online at kevinlwei.com or on Twitter at @kevinlwei

Feature requests, project collabs, want to get in touch? Shoot me an email at hi@kevinlwei.com

## License

This notebook is provided to you under the MIT License: https://opensource.org/licenses/MIT

Copyright (c) 2020 Kevin Wei

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Import Data

<b>Data Sources</b>: <br>
CFB contribution, expenditure, and metrics data available at: http://www.nyccfb.info/follow-the-money/data-library/ <br>
CFB list of 2021 NYC candidates available at: https://www.nyccfb.info/follow-the-money/candidates/ <br>
Doing Business database available at: https://www1.nyc.gov/dbnyc/index.htm <br>
NYSBOE Voterfile is available at: https://www.elections.ny.gov/FoilRequests.html

Note: metrics data has a time lag, so we don't use it for anything here

In [1]:
# import some packages
import pandas as pd
import numpy as np
import operator

In [2]:
# change viewing options to show entire dataframe when applicable
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 200)

In [3]:
# import data from CSV
contributions = pd.read_csv('CFB_All_Contributions.csv', sep=',', dtype={'RECIPID':str}, parse_dates=['DATE','REFUNDDATE'])
expenditures = pd.read_csv('CFB_All_Expenditures.csv', sep=',')
candidates = pd.read_csv('CFB_Candidate_List.csv', sep=',')
cfb_metrics = pd.read_csv('CFB_Financial_Analysis.csv', sep=',')
dbdb = pd.read_csv('DBDB_Download.csv', sep=',', parse_dates=['Doing Business Start Date', 'Run Date'])

/usr/local/Cellar/jupyterlab/2.1.5/libexec/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# import NYS State voterfile. Separate cell due to long runtime
# not currently in use; waiting on CFB to see if I can get address data for 2021 contributors
#voters = pd.read_csv('DemWorBlk.txt', sep=',', encoding='unicode_escape', names=['ln', 'fn', 'mn', 'suffix', 'r_add', 'r_halfcode', 'r_apt', 'r_predir', 'r_street', 'r_city', 'r_zip5', 'r_zip4', 'mail_add1', 'mail_add2', 'mail_add3', 'mail_add4', 'dob', 'gender', 'enrollment', 'other_party', 'county', 'ed', 'ld', 'towncity', 'ward', 'cd', 'sd', 'ad', 'last_vote_date', 'prev_year_voted', 'prev_county', 'prev_address', 'prev_name', 'county_vr_id', 'reg_date', 'vr_source', 'id_req', 'id_met', 'status', 'status_reason', 'inact_date', 'purge_date', 'sboe_id', 'voter_history'])

## Data Pre-Processing

### Functions for Data Pre-Processing

In [5]:
# function to trim every cell in string fields; to be applied before any string field dtype conversions
def trim_all(dfs):
    for i in dfs:
        i[i.select_dtypes(['object']).columns] = i.select_dtypes(['object']).apply(lambda x: x.str.strip())

In [6]:
# generate zip5 (int field) from zip column (string field)
def get_zip5(s):
    zip5 = s
    
    #eliminate zip4 or other appended data to zip (some of data is in format #####-###, which is invalid)
    if s.find('-') > -1:
        zip5 = s[:s.find('-')]
        
    #drops non-numeric values and blank zips, else int-formatted zip5
    if not zip5.isnumeric() or len(zip5) == 0:
        return np.nan
    else:
        return int(zip5)

In [7]:
# add column to candidate df with IDs
# IDs are also in the metrics table, but that table currently lags a filing period due to process time by CFB
def get_candidate_ids(check_ids=False):
    for i, row in candidates.iterrows():
        id_frequency = contributions[contributions['RECIPNAME'] == row['Candidate']]['RECIPID'].value_counts()
        
        #Print candidates with no ID or with more than 1 ID
        if check_ids == True and ((len(id_frequency) > 1) or len(id_frequency) == 0):
            print("-------------------")
            print(len(id_frequency))
            print(id_frequency)
            print(row['Candidate'])
            print("-------------------")
        
        if len(id_frequency) == 0:
            candidates.loc[candidates.index[candidates['Candidate'] == row['Candidate']],'ID'] = np.nan
        else:  
            candidates.loc[candidates.index[candidates['Candidate'] == row['Candidate']],'ID'] = str(max(id_frequency.iteritems(), key = operator.itemgetter(1))[0])

In [8]:
# processes special campaign cases from the candidate list (small campaigns, terminated campaigns)
def process_special_campaigns():
    candidates['Small Campaign'] = False
    candidates['Terminated'] = False
    
    for i, row in candidates.iterrows():
        if '*' in row['Candidate']:
            candidates.loc[i, 'Terminated'] = True
            candidates.loc[i, 'Candidate'] = candidates.loc[i, 'Candidate'].replace('*', '')
        if '§' in row['Candidate']:
            candidates.loc[i, 'Small Campaign'] = True
            candidates.loc[i, 'Candidate'] = candidates.loc[i, 'Candidate'].replace('§', '')

### Pre-Processing Data

In [9]:
# trim all data, process special campaign cases, and append candidate IDs to the candidate table
trim_all([contributions, expenditures, candidates, cfb_metrics])
process_special_campaigns()
get_candidate_ids()

In [10]:
# drop campaigns that have received no contributions from the candidates table (i.e., id = NaN)
candidates.dropna(subset=['ID'], inplace=True)

In [11]:
# standardize employer names/addresses, names, intermediary names/addresses

In [12]:
# match contributors to voterfile

In [13]:
# append doing business data to contributor list
# note: only individual contributors tested

## Data Analysis

### Functions for Data Analysis

In [14]:
# returns all rows of contributions for a particular candidate from the contributions table
# excludes refunds by default
def get_contributions(id, refunds=True):
    if id not in candidates['ID'].values:
        raise ValueError("Not a valid candidate ID. See col 'ID' in candidates table for valid ID options")
        
    # if refunds = True, return contribution list with refunds; else exclude refunds
    if refunds:
        return contributions[contributions['RECIPID'] == id]
    else:
        return contributions[(contributions['RECIPID'] == id) & (contributions['AMNT'] > 0)]

In [15]:
def get_unique_contributors(id):
    if id not in candidates['ID'].values:
        raise ValueError("Not a valid candidate ID. See col 'ID' in candidates table for valid ID options")
    else:
        data = get_contributions(id)
        
    return data['NAME'].str.lower().unique()

In [16]:
# returns all rows of expenditures for a particular candidate from the expenditures table
def get_expenditures(id, refunds=True):
    if id not in candidates['ID'].values:
        raise ValueError("Not a valid candidate ID. See col 'ID' in candidates table for valid ID options")
        
    # if refunds = True, return expenditure list with refunds; else exclude refunds
    if refunds:
        return expenditures[expenditures['CANDID'] == id]
    else:
        return expenditures[(expenditures['CANDID'] == id) & (expenditures['AMNT'] > 0)]

In [17]:
# returns contribution types breakdown (including db from individual contributions)
def get_contribution_types(data=None, id=None):
    if data is None and id is None:
        raise ValueError('Must pass either a pandas dataframe containing contributions or candidate id')
    if id != None:
        data = get_contributions(id)
    breakdown = {}
    types = {'candidate': 'CAN',
            'corporation': 'CORP',
            'labor_union': 'EMPO',
            'candidate_family': 'FAM',
            'individual': 'IND',
            'llc': 'LLC',
            'other': 'OTHR',
            'partnership': 'PART',
            'candidate_committee': 'PCOMC',
            'pac': 'PCOMP',
            'party': 'PCOMZ',
            'candidate_spouse': 'SPO',
            'unknown': 'UNKN'
            }
    
    for key, value in types.items():
        breakdown[key] = data[data['C_CODE'].str.contains(value, case=False, na=False)]
    
    return breakdown

In [18]:
# returns a dict separating contributions by contributor borough contribution location breakdown
def get_contribution_locations(data=None, id=None):
    if data is None and id is None:
        raise ValueError('Must pass either a pandas dataframe containing contributions or candidate id')
    if id != None:
        data = get_contributions(id)
    breakdown = {}
    types = {'manhattan': ['m'], 
             'bronx': ['x'], 
             'queens': ['q'], 
             'staten_island': ['s'], 
             'brooklyn': ['k'], 
             'outside_nyc': ['z'], 
             'all_nyc': ['m', 'k', 'x', 'q', 's']}
    
    for key, value in types.items():
        breakdown[key] = data[data['BOROUGHCD'].str.lower().isin(value)]
    
    return breakdown

In [19]:
# returns (estimated) contributions from specified industry
# can either pass in the data directly or a candidate ID; note passing in candidate ID will override data
# options for industry: 'law_enforcement', 'education', 'finance', 'healthcare', 'retired', 'real_estate'

# Law enforcement: includes any police department, corrections facilities/officers, US military
# Education: includes all students, educators, and any individual employed by an educational institution or NYCDOE
# Health care: includes all medical practicioners and adjacent professions, and all individuals employed by healthcare and public health institutions. Excludes individuals in the health insurance or health law industries
# Retired: includes all individuals who self-identified as retired
# Student: includes all individuals who self-identified as students
# Finance: includes all bankers, venture capitalists, traders, financial advisors, and all individuals employed by a bank, investment bank, private equity firm, or venture capital firm

def get_contribution_industries(data=None, id=None):
    if data is None and id is None:
        raise ValueError('Must pass either a pandas dataframe containing contributions or candidate id')
    if id != None:
        data = get_contributions(id)
    breakdown = {}
    types = {'law_enforcement': '(NYPD)|(military)|(police)|((?<!Salvation )army)|(air force)|(?<!Old )navy(?! yard)|(marine corps)|(correction)|(court officer)|(NYCDOC)|(NYC-DOC)|(NYC DOC)|(DOCC)|(^DOD$)|(of Defense)|(correction officer)|(sheriff)',
             'education': '(academy)|(DOE)|(Board of Education)|(school(?! construction authority))|(student)|(teacher)|(professor)|(university)|(college)',
             'finance': 'capital|(?<!ad)venture|(?<!reserve )(?<!coldwell)(?<!mil)(?<!world)bank(?! street)|goldman|morgan(?! lewis)(?! library)|lynch|barclays|UBS |^UBS$|bridgewater|two sigma|elliott management|blackrock|D[. ]*E[. ]*Shaw|investment bank|(?<!coldwell)(?<!coldwell )banker|^investment bank|financial advis[oe]r|hedge fund(?!s care)|private equity|private equity|trader(?! joe)',
             'healthcare': '(nurse(?!ry))|(physician)|((?<!post)(?<!post )(?<!post-)doctor)|(hospital(?!ity))|((?<!massage )(?<!massage)therapist)|(medical)|^RN$|^MD$|(DOHMH)|(NY[CS]+[ ]*DOH)|(NYC[ ]*H&H)|((?<!energizing )health(?! insurance)(?!insurance)(?! plan)(?!plan)(?!care)(?! care)(?! crisis)(?! law)(?!y))|(dental)|(dentist)|(pediatric)|(cardiolog)|(surgeon)|(surgery)|(dermatolog)|(neurolog)|(ophthalmolog)|(oncolog)|(patholog)|(physiatrict)|(psychiatr)|(radiolog)',
             'retired': '^[ ]*retired[ ]*$',
             'student': '^[ ]*student[ ]*$'
            }
    
    for key, value in types.items():
        breakdown[key] = data[(data['EMPNAME'].str.contains(value, case=False, na=False)) | (data['OCCUPATION'].str.contains(value, case=False, na=False))]
    
    return breakdown 

In [20]:
#returns expenditure types breakdown
def get_expenditure_types(data=None, id=None):
    if data is None and id is None:
        raise ValueError('Must pass either a pandas dataframe containing expenditures or candidate id')
    if id != None:
        data = get_expenditures(id)
    breakdown = {}
    types = {'advance_repayment': 'ADVAN',
            'mailers': 'CMAIL',
            'misc': 'CMISC',
            'political_contrib': 'CONTRB',
            'compliance': 'COMPL',
            'consulting': 'CONSL',
            'constituent_services': 'CONSV',
            'fundraising': 'FUNDR',
            'interest': 'INTER',
            'literature': 'LITER',
            'non_qualified_expenditures': 'NQUAL',
            'office': 'OFFCE',
            'other': 'OTHER',
            'petitioning': 'PETIT',
            'polling': 'POLLS',
            'postage': 'POSTA',
            'printing': 'PRINT',
            'professional_services': 'PROFL',
            'radio': 'RADIO',
            'tv': 'TVADS',
            'unknown': 'UNKN',
            'voter_reg': 'VOTER',
            'wages': 'WAGES'
            }
    
    for key, value in types.items():
        breakdown[key] = data[data['PURPOSECD'].str.contains(value, case=False, na=False)]
    
    return breakdown 

In [21]:
# returns expenditure location breakdown
def get_expenditure_locations(data=None, id=None):
    if data is None and id is None:
        raise ValueError('Must pass either a pandas dataframe containing expenditures or candidate id')
    if id != None:
        data = get_expenditures(id)
    breakdown = {}
    types = {'brooklyn': ['brooklyn'],
             'queens': ['queens', 'far rockaway', 'ozone park', 'woodside', 'howard beach', 'college pt', 'ridgewood', 'long island city', 'astoria', 'forest hills', 'rego park', 'jamaica', 'flushing', 'east elmuhrst', 'east elmhurst', 'bayside', 'maspeth', 'whitestone', 'jackson heights', 'sunnyside', 'little neck'],
             'bronx': ['bronx', 'the bronx'],
             'manhattan': ['new york', 'nueva york'],
             'staten_island': ['staten island', 'staten islan4'],
            }
    types['all_nyc'] = types['brooklyn'] + types['queens'] + types['bronx'] + types['manhattan'] + types['staten_island']
    
    for key, value in types.items():
        breakdown[key] = data[data['CITY'].str.lower().isin(value)]
    
    breakdown['outside_nyc'] = data[~data['CITY'].str.lower().isin(value)]
    
    return breakdown

In [22]:
# takes a dictionary as input, and returns a dict with the same keys but length of the original values

def get_type_count(dict):
    count = {}
    
    for key, values in dict:
        count[key] = len(values)
        
    return count

In [23]:
# takes a dictionary as input, and returns a dict with the same keys but the sum of the contributions as values

def get_type_sum(dict):
    count = {}
    
    for key, values in dict:
        count[key] = values['AMNT'].sum()
        
    return count

### Generating candidate-level metrics


In [24]:
# generated a MultiIndex object that will be used as column names in the final metrics df
col_names = [['Overview'] * 12 
         + ['Contribution Types: Raw Amounts'] * 13 
         + ['Contribution Types: % of Total Contributions'] * 13 
         + ['Contribution Types: Contributors'] * 13
         + ['Contribution Locations: Raw Amounts'] * 7
         + ['Contribution Locations: % of Total Contributions'] * 7
         + ['Contribution Locations: Contributors'] * 7
         + ['Contribution Industries: Raw Amounts'] * 6
         + ['Contribution Industries: % of Total Contributions'] * 6
         + ['Contribution Industries: Contributors'] * 6
         + ['Expenditure Locations: Raw Amounts'] * 7
         + ['Expenditure Locations: % of Total Expenditures'] * 7
         + ['Expenditure Types: Raw Amounts'] * 23
         + ['Expenditure Types: % of Total Expenditures'] * 23,
         ['Candidate', 'Public Financing Status', 'Office', 'Option', 'ID', 'Small Campaign', 'Terminated', 'Total Amount Raised', 'Total Contributors', 'Average Contribution', 'Total Expenditures', 'Cash On Hand'] 
         + ['candidate', 'corporation', 'labor_union', 'candidate_family', 'individual', 'llc', 'other', 'partnership', 'candidate_committee', 'pac', 'party', 'candidate_spouse', 'unknown'] * 3
         + ['manhattan', 'bronx', 'queens', 'staten_island', 'brooklyn', 'outside_nyc', 'all_nyc'] * 3
         + ['law_enforcement', 'education', 'finance', 'healthcare', 'retired', 'student'] * 3
         + ['manhattan', 'bronx', 'queens', 'staten_island', 'brooklyn', 'outside_nyc', 'all_nyc'] * 2
         + ['advance_repayment', 'mailers', 'misc', 'political_contrib', 'compliance', 'consulting', 'constituent_services', 'fundraising', 'interest', 'literature', 'non_qualified_expenditures', 'office', 'other', 'petitioning', 'polling', 'postage', 'printing', 'professional_services', 'radio', 'tv', 'unknown', 'voter_reg', 'wages'] * 2
        ]

col_index = pd.MultiIndex.from_arrays(col_names)

In [25]:
# create new df to store candidate fundraising metrics
candidate_metrics = pd.DataFrame(columns=col_index)

# copy over the overview data from the candidates table
for i in candidates.columns:
    candidate_metrics.loc[:,('Overview', i)] = candidates[i]

In [26]:
for i, row in candidate_metrics.iterrows():
    # get overview contribution/expenditure rows per candidate
    all_contributions = get_contributions(row[('Overview', 'ID')])
    #all_contributions_excluding_refunds = get_contributions(row[('Overview', 'ID')],refunds=False)
    uq_contributions = get_unique_contributors(row[('Overview', 'ID')])
    
    all_expenditures = get_expenditures(row[('Overview', 'ID')])
    #all_expenditures_excluding_refunds = get_expenditures(row[('Overview', 'ID')], refunds=False)
    
    # populate columns in overview metrics
    if all_contributions.empty or (uq_contributions.size == 0):
        candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] = 0
        candidate_metrics.loc[i, ('Overview', 'Total Contributors')] = 0
        candidate_metrics.loc[i, ('Overview', 'Average Contributions')] = 0
    else:
        candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] = all_contributions['AMNT'].sum()
        candidate_metrics.loc[i, ('Overview', 'Total Contributors')] = len(uq_contributions)
        candidate_metrics.loc[i, ('Overview', 'Average Contribution')] = round(float(candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')]) / float(candidate_metrics.loc[i, ('Overview', 'Total Contributors')]), 2)
    
    if all_expenditures.empty:
        candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] = 0
    else:
        candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] = all_expenditures['AMNT'].sum()
    
    candidate_metrics.loc[i, ('Overview', 'Cash On Hand')] = candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] - candidate_metrics.loc[i, ('Overview', 'Total Expenditures')]
    
    # these for loops below can *probably* be refactored to something simpler / more readable
    # get contribution types, locations, and industries
    contribution_types = get_contribution_types(all_contributions)
    contribution_locations = get_contribution_locations(all_contributions)
    contribution_industries = get_contribution_industries(all_contributions)
    
    # populate columns in contribution type metrics
    for j, k in contribution_types.items():
        if k.empty:
            candidate_metrics.loc[i, ('Contribution Types: Raw Amounts', j)] = 0
        else:
            candidate_metrics.loc[i, ('Contribution Types: Raw Amounts', j)] = k['AMNT'].sum()
            
        if candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] == 0:
            candidate_metrics.loc[i, ('Contribution Types: % of Total Contributions', j)] = '0%'
        else: 
            candidate_metrics.loc[i, ('Contribution Types: % of Total Contributions', j)] = str(round(candidate_metrics.loc[i, ('Contribution Types: Raw Amounts', j)] / candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] * 100, 2)) + '%'
        candidate_metrics.loc[i, ('Contribution Types: Contributors', j)] = len(k['NAME'].unique())
        
    # populate columns in contribution location metrics
    for j, k in contribution_locations.items():
        if k.empty:
            candidate_metrics.loc[i, ('Contribution Locations: Raw Amounts', j)] = 0
        else:
            candidate_metrics.loc[i, ('Contribution Locations: Raw Amounts', j)] = k['AMNT'].sum()
        if candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] == 0:
            candidate_metrics.loc[i, ('Contribution Locations: % of Total Contributions', j)] = '0%'
        else: 
            candidate_metrics.loc[i, ('Contribution Locations: % of Total Contributions', j)] = str(round(candidate_metrics.loc[i, ('Contribution Locations: Raw Amounts', j)] / candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] * 100, 2)) + '%'
        candidate_metrics.loc[i, ('Contribution Locations: Contributors', j)] = len(k['NAME'].unique())
   
    # populate columns in contribution industry metrics
    for j, k in contribution_industries.items():
        if k.empty:
            candidate_metrics.loc[i, ('Contribution Industries: Raw Amounts', j)] = 0
        else:
            candidate_metrics.loc[i, ('Contribution Industries: Raw Amounts', j)] = k['AMNT'].sum()
            
        if candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] == 0:
            candidate_metrics.loc[i, ('Contribution Industries: % of Total Contributions', j)] = '0%'
        else: 
            candidate_metrics.loc[i, ('Contribution Industries: % of Total Contributions', j)] = str(round(candidate_metrics.loc[i, ('Contribution Industries: Raw Amounts', j)] / candidate_metrics.loc[i, ('Overview', 'Total Amount Raised')] * 100, 2)) + '%'
        candidate_metrics.loc[i, ('Contribution Industries: Contributors', j)] = len(k['NAME'].unique())
    
    # get expenditure location and types
    # all expenditures for locations to capture refunds accurately for NYC vs non-NYC expenditures
    expenditure_locations = get_expenditure_locations(all_expenditures)
    expenditure_types = get_expenditure_types(all_expenditures)
    
    # populate columns in expenditure location metrics
    for j, k in expenditure_locations.items():
        if k.empty:
            candidate_metrics.loc[i, ('Expenditure Locations: Raw Amounts', j)] = 0
        else:
            candidate_metrics.loc[i, ('Expenditure Locations: Raw Amounts', j)] = k['AMNT'].sum()
            
        if candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] == 0:
            candidate_metrics.loc[i, ('Expenditure Locations: % of Total Expenditures', j)] = '0%'
        else: 
            candidate_metrics.loc[i, ('Expenditure Locations: % of Total Expenditures', j)] = str(round(candidate_metrics.loc[i, ('Expenditure Locations: Raw Amounts', j)] / candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] * 100, 2)) + '%'
   
    # populate columns in expenditure type metrics
    for j, k in expenditure_types.items():
        if k.empty:
            candidate_metrics.loc[i, ('Expenditure Types: Raw Amounts', j)] = 0
        else:
            candidate_metrics.loc[i, ('Expenditure Types: Raw Amounts', j)] = k['AMNT'].sum()
            
        if candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] == 0:
            candidate_metrics.loc[i, ('Expenditure Types: % of Total Expenditures', j)] = '0%'
        else: 
            candidate_metrics.loc[i, ('Expenditure Types: % of Total Expenditures', j)] = str(round(candidate_metrics.loc[i, ('Expenditure Types: Raw Amounts', j)] / candidate_metrics.loc[i, ('Overview', 'Total Expenditures')] * 100, 2)) + '%'

/usr/local/Cellar/jupyterlab/2.1.5/libexec/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


## Export Data

In [27]:
candidate_metrics.to_csv('2021 NYC Candidate Metrics.csv')